In [112]:
from credentials import my_SteamID, my_APIkey
import numpy as np
import pandas as pd
import urllib
import json

In [138]:


def get_friend_IDs(steamid, APIkey):
    getList_URL = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={APIkey}&steamid={steamid}&relationship=friend"
    file = urllib.request.urlopen(getList_URL)
    content = file.read().decode('utf-8')
    friendlist = json.loads(content)
    friendlist = pd.DataFrame(friendlist["friendslist"]["friends"])
    return friendlist["steamid"]

def get_friend_names(friend_IDs, APIkey):
    keys=["steamid", "personaname"]
    out=[]
    for id in friend_IDs:
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urllib.request.urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            out.append([friendstatus.get(key) for key in keys])
        except:
            continue
    return pd.DataFrame(out, columns=["steamid", "steamname"]).sort_values("steamname")

def fill_empty_ids(favs, friend_names):
    favs = pd.merge(favs, friend_names, on="steamname", suffixes=("","_2"))
    favs["steamid"]= favs["steamid_2"]
    favs.drop("steamid_2", axis=1, inplace=True)
    favs.to_csv("my_favorites.csv", sep=";", index=False)
    return favs

def check_fav_status(favs, APIkey):
    notify={}
    keys=["personaname", "steamid", "gameextrainfo"]
    for n in range(favs.shape[0]):
        id= favs.iloc[n]["steamid"]
        games= [x.strip() for x in favs.iloc[n]["games"].split(",")]
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urllib.request.urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            name = friendstatus["personaname"]
            try:
                game = friendstatus["gameextrainfo"]
                if game in games:
                    notify[name] = game
            except: 
                notify[name] = ""
        except:
            continue
    return notify


def check_for_change(hits, notify):
    send = False
    for entry in hits.keys():
        if notify[entry] != hits[entry]:
            notify[entry] = hits[entry]
            if notify[entry] != "":
                send = True
    return notify, send





In [132]:
# use credentials to get steamID of entire friendlist
my_friend_IDs = get_friend_IDs(my_SteamID, my_APIkey)

# look up the steam names for all IDs in friendlist
my_friend_names = get_friend_names(my_friend_IDs, my_APIkey)

#load table of favorites
my_favs = pd.read_csv("my_favorites.csv", delimiter=";")

# fill empty IDs in favorites
my_favs = fill_empty_ids(my_favs, my_friend_names)

In [131]:
my_hits = check_fav_status(my_favs, my_APIkey)
my_hits

Pail ist nicht online
Smallboy_Slin ist nicht online
[WC] Scott Sterling spielt Sands of Salzaar


{'Pail': '', 'Smallboy_Slin': '', '[WC] Scott Sterling': 'Sands of Salzaar'}

In [ ]:
# check if entries in notify are the same as in current_hits
# if not, replace it and return true
my_notify

{'Pail': '', 'Smallboy_Slin': '', '[WC] Scott Sterling': 'Sands of Salzaar'}

{'Pail': 'Warhammer: Vermintide 2', 'Smallboy_Slin': '', '[WC] Scott Sterling': 'Sands of Salzaar'}
False


In [143]:
# create empty dict for notifications
my_notify = {}
for i in range(my_favs.shape[0]):
    my_notify[my_favs.iloc[i]["steamname"]]=""
my_notify

x=0
while x<100:
    my_hits = check_fav_status(my_favs, my_APIkey)
    my_notify, my_send = check_for_change(my_hits, my_notify)
    if my_send == True:
        for i in my_notify.keys():
            if my_notify[i] != "":
                print(f"{i} is playing {my_notify[i]}")
    else:
        print(".")
    time.sleep(10)
    x +=1

Pail is playing Warhammer: Vermintide 2
[WC] Scott Sterling is playing Sands of Salzaar
.
.
.
.
.
Pail is playing Warhammer: Vermintide 2
[WC] Scott Sterling is playing Sands of Salzaar
.
.


KeyboardInterrupt: 

In [ ]:
import time

In [ ]:
def get_ingame_friends(friendlist, APIkey):
    keys=["steamid", "personaname", "gameextrainfo"]
    out=[]
    for id in friendlist["steamid"]:
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urllib.request.urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            if friendstatus["gameextrainfo"] != None:
                out.append([friendstatus.get(key) for key in keys])
        except:
            continue
    return pd.DataFrame(out, columns=keys)

In [ ]:
my_ingame_friends = get_ingame_friends(my_friendlist, my_APIkey)
my_ingame_friends

NameError: name 'my_friendlist' is not defined

In [ ]:
# online friends
my_friend_info[my_friend_info["personastate"] != 0].sort_values("personastate")

NameError: name 'my_friend_info' is not defined

In [ ]:
id= "76561198026715831"
getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={my_APIkey}&steamids={id}"
file = urllib.request.urlopen(getFriend_URL)
content = file.read().decode('utf-8')
friendstatus = json.loads(content)
friendstatus

{'response': {'players': [{'steamid': '76561198026715831',
    'communityvisibilitystate': 3,
    'profilestate': 1,
    'personaname': '[WC] Scott Sterling',
    'profileurl': 'https://steamcommunity.com/id/socing/',
    'avatar': 'https://avatars.akamai.steamstatic.com/c18e067d5c5b0654369a42ae92a32ac105f42e21.jpg',
    'avatarmedium': 'https://avatars.akamai.steamstatic.com/c18e067d5c5b0654369a42ae92a32ac105f42e21_medium.jpg',
    'avatarfull': 'https://avatars.akamai.steamstatic.com/c18e067d5c5b0654369a42ae92a32ac105f42e21_full.jpg',
    'avatarhash': 'c18e067d5c5b0654369a42ae92a32ac105f42e21',
    'lastlogoff': 1673943546,
    'personastate': 3,
    'realname': 'Scott Sterling',
    'primaryclanid': '103582791463890108',
    'timecreated': 1276860108,
    'personastateflags': 0,
    'gameextrainfo': 'Sands of Salzaar',
    'gameid': '1094520',
    'loccountrycode': 'BD'}]}}

In [ ]:
id= "76561198059254917"
getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={my_APIkey}&steamids={id}"
file = urllib.request.urlopen(getFriend_URL)
content = file.read().decode('utf-8')
friendstatus = json.loads(content)
friendstatus

{'response': {'players': [{'steamid': '76561198059254917',
    'communityvisibilitystate': 3,
    'profilestate': 1,
    'personaname': 'Pail',
    'commentpermission': 2,
    'profileurl': 'https://steamcommunity.com/id/Pail_1/',
    'avatar': 'https://avatars.akamai.steamstatic.com/eef75511ba1f16b66e9ac45346e7c0f790a6551a.jpg',
    'avatarmedium': 'https://avatars.akamai.steamstatic.com/eef75511ba1f16b66e9ac45346e7c0f790a6551a_medium.jpg',
    'avatarfull': 'https://avatars.akamai.steamstatic.com/eef75511ba1f16b66e9ac45346e7c0f790a6551a_full.jpg',
    'avatarhash': 'eef75511ba1f16b66e9ac45346e7c0f790a6551a',
    'lastlogoff': 1673887105,
    'personastate': 0,
    'realname': 'Schniin',
    'primaryclanid': '103582791429521408',
    'timecreated': 1329986845,
    'personastateflags': 0,
    'loccountrycode': 'DE'}]}}

In [ ]:
df

,Freund,SteamName,steamid,Games
0,Schniin,Pail,NaN,Warhammer: Vermintide 2
1,Nils,Smallboy_Slin,NaN,"Deep Rock Galactic,SCUM"


In [ ]:
def check_fav_status(favs, APIkey):
    keys=["personaname", "steamid", "gameextrainfo"]
    out=[]
    for id in favs["steamid"]:
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urllib.request.urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            if friendstatus["gameextrainfo"] != None:
                out.append([friendstatus.get(key) for key in keys])
        except:
            continue
    return pd.DataFrame(out, columns=["steamname", "steamid", "gameextrainfo"])